In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#import libraries
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division
from sklearn.cluster import KMeans

In [0]:
pip install plotly

In [0]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objects as go
import plotly.offline as pyoff

Read Data from the csv

In [0]:
tx_data = pd.read_excel(r'/content/drive/My Drive/ADM/Project/rfm/twi_ds.xlsx')
#tx_data['start_time'] = pd.to_datetime(tx_data['start_time'])
#tx_us = tx_data.query("country=='United States'").reset_index(drop=True)

In [0]:
#convert the string date field to datetime
import datetime as dt
tx_data['start_time'] = pd.to_datetime(tx_data['start_time']).dt.date

#we will be using only UK data
tx_us = tx_data.query("country=='United States'").reset_index(drop=True)

In [0]:
tx_us = tx_us[pd.notnull(tx_us['viewer_id'])]

In [0]:
tx_us = tx_us[(tx_us['duration']>0)]

In [0]:
tx_us.head()

,visit_id,start_time,viewer_id,duration,country
0,536945,2018-12-03,14083.0,7.95,United States
1,536945,2018-12-03,14083.0,7.95,United States
2,536945,2018-12-03,14083.0,15.90,United States
3,536945,2018-12-03,14083.0,16.95,United States
4,536945,2018-12-03,14083.0,3.75,United States


In [0]:
#create 3m and 6m dataframes
tx_3m = tx_us[(tx_us.start_time < date(2019,6,1)) & (tx_us.start_time >= date(2019,3,1))].reset_index(drop=True)
tx_6m = tx_us[(tx_us.start_time >= date(2019,6,1)) & (tx_us.start_time < date(2019,12,1))].reset_index(drop=True)

In [0]:
tx_3m

,visit_id,start_time,viewer_id,duration,country
0,545220,2019-03-01,14620.0,15.90,United States
1,545220,2019-03-01,14620.0,15.90,United States
2,545220,2019-03-01,14620.0,19.80,United States
3,545220,2019-03-01,14620.0,14.85,United States
4,545220,2019-03-01,14620.0,15.00,United States
...,...,...,...,...,...
69516,555133,2019-05-31,16810.0,14.85,United States
69517,555133,2019-05-31,16810.0,18.60,United States
69518,555133,2019-05-31,16810.0,8.50,United States
69519,555133,2019-05-31,16810.0,20.80,United States


In [0]:
#create tx_user for assigning clustering
tx_user = pd.DataFrame(tx_3m['viewer_id'].unique())
tx_user.columns = ['viewer_id']

In [0]:
#order cluster method
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

Recency

In [0]:
#calculate recency score
tx_max_purchase = tx_3m.groupby('viewer_id').start_time.max().reset_index()
tx_max_purchase.columns = ['viewer_id','MaxPurchaseDate']
tx_max_purchase['Recency'] = (tx_max_purchase['MaxPurchaseDate'].max() - tx_max_purchase['MaxPurchaseDate']).dt.days
tx_user = pd.merge(tx_user, tx_max_purchase[['viewer_id','Recency']], on='viewer_id')

kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Recency']])
tx_user['RecencyCluster'] = kmeans.predict(tx_user[['Recency']])

tx_user = order_cluster('RecencyCluster', 'Recency',tx_user,False)


In [0]:
tx_user.groupby('RecencyCluster')['Recency'].describe()

,count,mean,std,min,25%,50%,75%,max
RecencyCluster,,,,,,,,
0,353.0,76.660057,8.387153,64.0,69.00,75.0,84.0,91.0
1,435.0,50.400000,7.205413,40.0,44.00,50.0,56.0,63.0
2,449.0,24.412027,4.967927,18.0,20.00,23.0,27.0,35.0
3,564.0,8.826241,4.485020,0.0,5.75,9.0,12.0,16.0


Frequency 

In [0]:

#calcuate frequency score
tx_frequency = tx_3m.groupby('viewer_id').start_time.count().reset_index()
tx_frequency.columns = ['viewer_id','Frequency']
tx_user = pd.merge(tx_user, tx_frequency, on='viewer_id')

kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Frequency']])
tx_user['FrequencyCluster'] = kmeans.predict(tx_user[['Frequency']])

tx_user = order_cluster('FrequencyCluster', 'Frequency',tx_user,True)

In [0]:
tx_user.groupby('FrequencyCluster')['Frequency'].describe()

,count,mean,std,min,25%,50%,75%,max
FrequencyCluster,,,,,,,,
0,1563.0,23.936020,17.259294,1.0,10.0,20.0,35.0,68.0
1,226.0,112.694690,43.495488,69.0,80.0,98.0,129.0,282.0
2,11.0,482.636364,114.820096,328.0,421.5,438.0,558.5,673.0
3,1.0,1331.000000,NaN,1331.0,1331.0,1331.0,1331.0,1331.0


Engagement

In [0]:
#calcuate egagement score
tx_3m['Engagement'] = tx_3m['duration']
tx_engagement = tx_3m.groupby('viewer_id').Engagement.sum().reset_index()
tx_user = pd.merge(tx_user, tx_engagement, on='viewer_id')

kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Engagement']])
tx_user['EngagementCluster'] = kmeans.predict(tx_user[['Engagement']])
tx_user = order_cluster('EngagementCluster', 'Engagement',tx_user,True)


In [0]:
tx_user.groupby('EngagementCluster')['Engagement'].describe()

,count,mean,std,min,25%,50%,75%,max
EngagementCluster,,,,,,,,
0,1529.0,399.555598,264.849790,2.90,196.2500,325.100,548.9000,1152.50
1,241.0,1895.813988,744.560049,1154.40,1361.3200,1622.480,2134.6700,4514.24
2,22.0,7283.577273,2029.230036,4867.90,6126.2125,6710.395,7985.0825,12382.32
3,9.0,20149.676667,6647.721889,14984.73,16296.2400,17393.540,21535.9000,35111.22


Overall Scoring

In [0]:
#overall scoring
tx_user['OverallScore'] = tx_user['RecencyCluster'] + tx_user['FrequencyCluster'] + tx_user['EngagementCluster']
tx_user['Segment'] = 'Low-Value'
tx_user.loc[tx_user['OverallScore']>2,'Segment'] = 'Mid-Value' 
tx_user.loc[tx_user['OverallScore']>4,'Segment'] = 'High-Value' 

In [0]:
tx_user.head()

,viewer_id,Recency,RecencyCluster,Frequency,FrequencyCluster,Engagement,EngagementCluster,OverallScore,Segment
0,14620.0,13,3,26,0,412.78,0,3,Mid-Value
1,18044.0,6,3,57,0,808.96,0,3,Mid-Value
2,18075.0,12,3,34,0,660.07,0,3,Mid-Value
3,15241.0,0,3,63,0,965.40,0,3,Mid-Value
4,15660.0,12,3,31,0,529.72,0,3,Mid-Value


Since our feature set is ready, let’s calculate 6 months LTV for each customer which we are going to use for training our model.

We are taking Enagagement for our LTV directly.


In [0]:
#calculate Engagement eate a new dataframe for it
tx_6m['Engagement'] = tx_6m['duration']
tx_user_6m = tx_6m.groupby('viewer_id')['Engagement'].sum().reset_index()
tx_user_6m.columns = ['viewer_id','m6_Engagement']


#plot LTV histogram
plot_data = [
    go.Histogram(
        x=tx_user_6m.query('m6_Engagement < 10000')['m6_Engagement']
    )
]

plot_layout = go.Layout(
        title='6m Engagement'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

We have some outliers too. Filtering out the outliers makes sense to have a proper machine learning model.
Ok, next step. We will merge our 3 months and 6 months dataframes to see correlations between LTV and the feature set we have.

In [0]:
tx_merge = pd.merge(tx_user, tx_user_6m, on='viewer_id', how='left')
tx_merge = tx_merge.fillna(0)

tx_graph = tx_merge.query("m6_Engagement < 30000")

tx_ltv_viz =  pd.DataFrame(tx_graph)
tx_ltv_viz.to_csv(r'/content/drive/My Drive/ADM/Project/rfm/tf_ltv_viz.csv') 

In [0]:


plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Low-Value'")['m6_Engagement'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Mid-Value'")['m6_Engagement'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'High-Value'")['m6_Engagement'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "6m LTV"},
        xaxis= {'title': "RFE Score"},
        title='LTV'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)
#st.plotly_chart(fig, use_container_width=True)

In [0]:
#remove outliers
tx_merge = tx_merge[tx_merge['m6_Engagement']<tx_merge['m6_Engagement'].quantile(0.99)]


#creating 3 clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(tx_merge[['m6_Engagement']])
tx_merge['LTVCluster'] = kmeans.predict(tx_merge[['m6_Engagement']])

#order cluster number based on LTV
tx_merge = order_cluster('LTVCluster', 'm6_Engagement',tx_merge,True)

#creatinga new cluster dataframe
tx_cluster = tx_merge.copy()

#see details of the clusters
tx_cluster.groupby('LTVCluster')['m6_Engagement'].describe()

,count,mean,std,min,25%,50%,75%,max
LTVCluster,,,,,,,,
0,1403.0,439.024206,459.858261,0.00,0.0000,314.38,751.285,1584.37
1,329.0,2731.147994,1017.180243,1592.20,1916.5200,2399.66,3310.900,5727.25
2,50.0,8781.678200,3048.230191,5798.88,6426.1425,7327.48,9986.340,17101.04


2 is the best with average 8.2k LTV whereas 0 is the worst with 396.

In [0]:
tx_cluster.head()

,viewer_id,Recency,RecencyCluster,Frequency,FrequencyCluster,Engagement,EngagementCluster,OverallScore,Segment,m6_Engagement,LTVCluster
0,14620.0,13,3,26,0,412.78,0,3,Mid-Value,0.00,0
1,18044.0,6,3,57,0,808.96,0,3,Mid-Value,994.49,0
2,18075.0,12,3,34,0,660.07,0,3,Mid-Value,1341.03,0
3,15241.0,0,3,63,0,965.40,0,3,Mid-Value,791.04,0
4,15660.0,12,3,31,0,529.72,0,3,Mid-Value,517.48,0


In [0]:
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split


In [0]:
#convert categorical columns to numerical
tx_class = pd.get_dummies(tx_cluster)

#calculate and show correlations
corr_matrix = tx_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

#create X and y, X will be feature set and y is the label - LTV
X = tx_class.drop(['LTVCluster','m6_Engagement'],axis=1)
y = tx_class['LTVCluster']

#split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=56)

In [0]:
#XGBoost Multiclassification Model
ltv_xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1,objective= 'multi:softprob',n_jobs=-1).fit(X_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(ltv_xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(ltv_xgb_model.score(X_test[X_train.columns], y_test)))

y_pred = ltv_xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

Accuracy of XGB classifier on training set: 0.93
Accuracy of XGB classifier on test set: 0.81
              precision    recall  f1-score   support

           0       0.91      0.93      0.92        74
           1       0.33      0.33      0.33        12
           2       0.00      0.00      0.00         4

    accuracy                           0.81        90
   macro avg       0.41      0.42      0.42        90
weighted avg       0.79      0.81      0.80        90



In [0]:
y_test

707     0
520     0
1211    0
1171    0
1294    0
       ..
1190    0
275     0
1540    1
1131    0
203     0
Name: LTVCluster, Length: 90, dtype: int64